In this notebook, we directly compare two models, GP-RBF/Clustering hybrid and the GP-RBF/Kalman hybrid, in the change point experiment as the LOO scores for both models were extremly close. As such, we want to calculate the standard error of the difference score to test whether they are statistically distinguishable. This requires re-estimating both models.

In [1]:
import numpy as np
import pandas as pd

import pymc3 as pm
from theano.tensor.nnet.nnet import softmax
from theano import tensor as tt
import theano as t

print('Runing on PyMC3 v{}'.format(pm.__version__))
print('Runing on Theano v{}'.format(t.__version__))

Runing on PyMC3 v3.4.1
Runing on Theano v1.0.2


/anaconda3/envs/gp_learning/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def construct_sticky_choice(raw_data, n_arms=8):
    x_sc = []
    for subj in set(raw_data['id']):
        y = pd.get_dummies(raw_data.loc[raw_data['id'] == subj, 'arm'])

        # so, not every one uses every response, so we need to correct for this
        for c in set(range(1, n_arms + 1)):
            if c not in set(y.columns):
                y[c] = np.zeros(len(y), dtype=int)
        y = y.values

        x_sc.append(np.concatenate([np.zeros((1, n_arms)), y[:-1, :]]))

    return np.concatenate(x_sc)


def construct_subj_idx(data_frame):
    subj_idx = []
    subjs_inc = {}
    for s in data_frame['id'].values:
        if s not in subjs_inc:
            subjs_inc[s] = len(subjs_inc)
        subj_idx.append(subjs_inc[s])
    subj_idx = np.array(subj_idx)
    return subj_idx


clustering_data = pd.read_pickle('Data/exp_changepoint/exp_cp_clustering_means_std.pkl')
clustering_data.index = range(len(clustering_data))

lin_gp_data = pd.read_csv('Data/exp_changepoint/changelinpred.csv')
lin_gp_data.index = range(len(lin_gp_data))

rbf_gp_data = pd.read_csv('Data/exp_changepoint/changerbfpred.csv')
rbf_gp_data.index = range(len(rbf_gp_data))

kalman_data = pd.read_csv('Data/exp_changepoint/changekalmanpred.csv')
kalman_data.index = range(len(kalman_data))

bayes_gp_data = pd.read_pickle('Data/exp_changepoint/bayes_gp_exp_cp.pkl')
bayes_gp_data.index = range(len(bayes_gp_data))

raw_data = pd.read_csv('Data/exp_changepoint/changepoint.csv', header=0)

# the GP-RBF can fail if subject always choose the same response. For simplicity, we are dropping those
# subjects
subjects_to_drop = set()
for s in set(raw_data.id):
    if s not in set(rbf_gp_data.id):
        subjects_to_drop.add(s)

for s in subjects_to_drop:
    clustering_data = clustering_data[clustering_data['Subject'] != s].copy()
    lin_gp_data = lin_gp_data[lin_gp_data.id != s].copy()
    raw_data = raw_data[raw_data.id != s].copy()
    kalman_data = kalman_data[kalman_data.id != s].copy()
    bayes_gp_data = bayes_gp_data[bayes_gp_data['Subject'] != s].copy()

# construct a sticky choice predictor. This is the same for all of the models
x_sc = construct_sticky_choice(raw_data)

# PYMC3 doesn't care about the actual subject numbers, so remap these to a sequential list
subj_idx = construct_subj_idx(lin_gp_data)
n_subj = len(set(subj_idx))

# prep the predictor vectors
x_mu_cls = np.array([clustering_data.loc[:, 'mu_%d' % ii].values for ii in range(8)]).T
x_sd_cls = np.array([clustering_data.loc[:, 'std_%d' % ii].values for ii in range(8)]).T

x_mu_bayes_gp = np.array([bayes_gp_data.loc[:, 'mu_%d' % ii].values for ii in range(8)]).T
x_sd_bayes_gp = np.array([bayes_gp_data.loc[:, 'std_%d' % ii].values for ii in range(8)]).T

x_mu_lin = np.array([lin_gp_data.loc[:, 'mu_%d' % ii].values for ii in range(8)]).T
x_sd_lin = np.array([lin_gp_data.loc[:, 'sigma_%d' % ii].values for ii in range(8)]).T

x_mu_rbf = np.array([rbf_gp_data.loc[:, 'mu_%d' % ii].values for ii in range(8)]).T
x_sd_rbf = np.array([rbf_gp_data.loc[:, 'sigma_%d' % ii].values for ii in range(8)]).T

x_mu_kal = np.array([kalman_data.loc[:, 'mu_%d' % ii].values for ii in range(8)]).T
x_sd_kal = np.array([kalman_data.loc[:, 'sig_%d' % ii].values for ii in range(8)]).T

y = raw_data['arm'].values - 1  # convert to 0 indexing


# print "Experiment Change Point, Running %d subjects" % model_matrix['n_subj']
# run_save_models(model_matrix, name_tag='exp_cp', sample_kwargs=sample_kwargs)

## Estimate GP-RBF/Clustering Model

In [3]:
n, d = x_mu_cls.shape
sample_kwargs = dict(draws=2000, njobs=2, tune=2000, init='advi+adapt_diag')

with pm.Model() as hier_rbf_clus:
    mu_1 = pm.Normal('mu_beta_rbf_mean', mu=0., sd=100.)
    mu_2 = pm.Normal('mu_beta_rbf_stdv', mu=0., sd=100.)
    mu_3 = pm.Normal('mu_beta_cls_mean', mu=0., sd=100.)
    mu_4 = pm.Normal('mu_beta_cls_stdv', mu=0., sd=100.)
    mu_5 = pm.Normal('mu_beta_stick',    mu=0., sd=100.)

    sigma_1 = pm.HalfCauchy('sigma_rbf_means', beta=100)
    sigma_2 = pm.HalfCauchy('sigma_rbf_stdev', beta=100)
    sigma_3 = pm.HalfCauchy('sigma_cls_means', beta=100)
    sigma_4 = pm.HalfCauchy('sigma_cls_stdev', beta=100)
    sigma_5 = pm.HalfCauchy('sigma_stick',     beta=100)

    b_1 = pm.Normal('beta_rbf_mu',  mu=mu_1, sd=sigma_1, shape=n_subj)
    b_2 = pm.Normal('beta_rbf_std', mu=mu_2, sd=sigma_2, shape=n_subj)
    b_3 = pm.Normal('beta_cls_mu',  mu=mu_3, sd=sigma_3, shape=n_subj)
    b_4 = pm.Normal('beta_cls_std', mu=mu_4, sd=sigma_4, shape=n_subj)
    b_5 = pm.Normal('beta_sc',      mu=mu_5, sd=sigma_5, shape=n_subj)

    rho = \
        tt.tile(tt.reshape(b_1[subj_idx], (n, 1)), d) * x_mu_rbf + \
        tt.tile(tt.reshape(b_2[subj_idx], (n, 1)), d) * x_sd_rbf + \
        tt.tile(tt.reshape(b_3[subj_idx], (n, 1)), d) * x_mu_cls + \
        tt.tile(tt.reshape(b_4[subj_idx], (n, 1)), d) * x_sd_cls + \
        tt.tile(tt.reshape(b_5[subj_idx], (n, 1)), d) * x_sc

    p_hat = softmax(rho)

    # Data likelihood
    yl = pm.Categorical('yl', p=p_hat, observed=y)

    # inference! (note: the progress bar may show up in the terminal window)
    trace_gprbf_cls = pm.sample(**sample_kwargs)


Auto-assigning NUTS sampler...
Initializing NUTS using advi+adapt_diag...
Average Loss = 41,183:  20%|██        | 40697/200000 [18:43<1:13:17, 36.23it/s]   
Convergence archived at 40700
Interrupted at 40,699 [20%]: Average Loss = 51,518
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [beta_sc, beta_cls_std, beta_cls_mu, beta_rbf_std, beta_rbf_mu, sigma_stick_log__, sigma_cls_stdev_log__, sigma_cls_means_log__, sigma_rbf_stdev_log__, sigma_rbf_means_log__, mu_beta_stick, mu_beta_cls_stdv, mu_beta_cls_mean, mu_beta_rbf_stdv, mu_beta_rbf_mean]


## Estimate GP-RBF/Kalman Model

In [4]:
with pm.Model() as hier_lin_clus:
    mu_1 = pm.Normal('mu_beta_lin_mean', mu=0., sd=100.)
    mu_2 = pm.Normal('mu_beta_lin_stdv', mu=0., sd=100.)
    mu_3 = pm.Normal('mu_beta_cls_mean', mu=0., sd=100.)
    mu_4 = pm.Normal('mu_beta_cls_stdv', mu=0., sd=100.)
    mu_5 = pm.Normal('mu_beta_stick',    mu=0., sd=100.)

    sigma_1 = pm.HalfCauchy('sigma_lin_means', beta=100)
    sigma_2 = pm.HalfCauchy('sigma_lin_stdev', beta=100)
    sigma_3 = pm.HalfCauchy('sigma_cls_means', beta=100)
    sigma_4 = pm.HalfCauchy('sigma_cls_stdev', beta=100)
    sigma_5 = pm.HalfCauchy('sigma_stick',     beta=100)

    b_1 = pm.Normal('beta_lin_mu',  mu=mu_1, sd=sigma_1, shape=n_subj)
    b_2 = pm.Normal('beta_lin_std', mu=mu_2, sd=sigma_2, shape=n_subj)
    b_3 = pm.Normal('beta_cls_mu',  mu=mu_3, sd=sigma_3, shape=n_subj)
    b_4 = pm.Normal('beta_cls_std', mu=mu_4, sd=sigma_4, shape=n_subj)
    b_5 = pm.Normal('beta_sc',      mu=mu_5, sd=sigma_5, shape=n_subj)

    rho = \
        tt.tile(tt.reshape(b_1[subj_idx], (n, 1)), d) * x_mu_lin + \
        tt.tile(tt.reshape(b_2[subj_idx], (n, 1)), d) * x_sd_lin + \
        tt.tile(tt.reshape(b_3[subj_idx], (n, 1)), d) * x_mu_cls + \
        tt.tile(tt.reshape(b_4[subj_idx], (n, 1)), d) * x_sd_cls + \
        tt.tile(tt.reshape(b_5[subj_idx], (n, 1)), d) * x_sc

    p_hat = softmax(rho)

    # Data likelihood
    yl = pm.Categorical('yl', p=p_hat, observed=y)

    # inference!
    trace_gplin_cls = pm.sample(**sample_kwargs)

Auto-assigning NUTS sampler...
Initializing NUTS using advi+adapt_diag...
Average Loss = 41,182:  21%|██▏       | 42596/200000 [24:39<1:31:08, 28.79it/s]   
Convergence archived at 42600
Interrupted at 42,599 [21%]: Average Loss = 51,093
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [beta_sc, beta_cls_std, beta_cls_mu, beta_lin_std, beta_lin_mu, sigma_stick_log__, sigma_cls_stdev_log__, sigma_cls_means_log__, sigma_lin_stdev_log__, sigma_lin_means_log__, mu_beta_stick, mu_beta_cls_stdv, mu_beta_cls_mean, mu_beta_lin_stdv, mu_beta_lin_mean]


## Directly Compare the two models using the Standard error of the LOO

In [5]:
df_comp_loo = pm.compare({hier_rbf_clus:trace_gprbf_cls, hier_lin_clus: trace_gplin_cls}, ic='LOO')

/anaconda3/envs/gp_learning/lib/python2.7/site-packages/pymc3/stats.py:292: UserWarning: Estimated shape parameter of Pareto distribution is
        greater than 0.7 for one or more samples.
        You should consider using a more robust model, this is because
        importance sampling is less likely to work well if the marginal
        posterior and LOO posterior are very different. This is more likely to
        happen with a non-robust model and highly influential observations.
  happen with a non-robust model and highly influential observations.""")


In [6]:
df_comp_loo

,LOO,pLOO,dLOO,weight,SE,dSE,shape_warn
1,81324.1,395.66,0,1,479.89,0,1
0,81325.8,396.37,1.76,0,479.86,0.82,1


In [58]:
-np.log2(np.exp(df_comp_loo['dLOO'].max() / -2))

1.2695716359822877